# Collecting and preparing data

In [1]:
import os, json, requests
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import cbsodata
from datetime import date, timedelta

with open('./data/sources.json', 'r') as json_file:
    resources = json.load(json_file)

base_dir = './data/csvs/'

def cleanup(df):
    for c in df.columns:
        if 'ersion' in c.lower():
            df.drop(columns=c, inplace=True)
    return df

## (Down)loading data

In [2]:
print('Pulling data from sources.json:')
for org,sets in resources.items():
    if org == 'CBS':
        for dataset,identifier in sets.items():
            name = '_'.join(dataset.lower().split())
            data_loc = base_dir + org.lower() + '_' + name + '.csv'
            try:
                os.stat(data_loc)
            except:
                pd.DataFrame(cbsodata.get_data(identifier)).to_csv(data_loc, index=False)
            finally:
                globals()[name] = pd.read_csv(data_loc)
            metadata = cbsodata.get_info(identifier)
            print('(Source: '+org+')  '+name+' @ '+'https://opendata.cbs.nl/statline/portal.html?_catalog=CBS&_la=nl&tableId='+identifier+'&_theme=76')
    else:
        for dataset,urls in sets.items():
            name = '_'.join(dataset.lower().split())
            data_loc = base_dir + org.lower() + '_' + name + '.csv'
            try:
                os.stat(data_loc)
            except:
                if urls['data'][-4:] == '.csv':
                    pd.read_csv(urls['data']).to_csv(data_loc, index=False)
                elif urls['data'][-5:] == '.json':
                    pd.read_json(urls['data']).to_csv(data_loc, index=False)
                else:
                    print('can not handle file format: just dopwnloading file')
                    data_loc = data_loc[-4:]+'.temp'
                    print(data_loc)
                    print(urls['data'])
                    req = requests.get(urls['data'], stream=True)
                    print(req)
                    with open(data_loc[:-5], 'wb') as f:
                        f.write(req.content)
            finally:
                globals()[name] = pd.read_csv(data_loc)
            print('(Source: '+org+') '+name+' @ '+urls['meta'])

Pulling data from sources.json:
(Source: CBS)  deceased @ https://opendata.cbs.nl/statline/portal.html?_catalog=CBS&_la=nl&tableId=70895ned&_theme=76
(Source: CBS)  population @ https://opendata.cbs.nl/statline/portal.html?_catalog=CBS&_la=nl&tableId=37230ned&_theme=76
(Source: RIVM) counts_per_municipality @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/5f6bc429-1596-490e-8618-1ed8fd768427
(Source: RIVM) prevalence @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/097155aa-75eb-4caa-8ed3-4c6edb80467e
(Source: RIVM) r @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/ed0699d1-c9d5-4436-8517-27eb993eab6e
(Source: RIVM) homes_for_elderly @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/0179dd26-7bf6-4021-857f-8623aaf8e73a
(Source: RIVM) homes_for_disabled @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/aa28b74d-7150-47bc-a7cb-98876e93381b
(Source: RIVM) seventy_plus_living_at_home @ https://data.riv

## Preparing data

In [3]:
print()
print('Creating dataframes for analysis:')


Creating dataframes for analysis:


### CBS
#### Deceased

In [4]:
# Totals only
if 'ID' in deceased.columns:
    deceased = deceased.drop(columns='ID')
if 'Geslacht' in deceased.columns:
    deceased = deceased.loc[deceased['Geslacht'] == 'Totaal mannen en vrouwen',:]
    deceased.drop(columns='Geslacht', inplace=True)
deceased = deceased.loc[deceased['LeeftijdOp31December'] != 'Totaal leeftijd',:]
deceased.rename(columns={'Overledenen_1':'Deceased'}, inplace=True)

#### Deceased per week

In [5]:
# weeks only, no totals for year
deceased_week_agegroup = deceased[deceased['Perioden'].str.contains('week')]
deceased_week_agegroup['year'] = deceased_week_agegroup['Perioden'].str.slice(stop=4).astype(int)

# split Perioden into year and week
deceased_week_agegroup['week'] = deceased_week_agegroup['Perioden'].str.slice(start=10,stop=12).astype(int)
deceased_week_agegroup = deceased_week_agegroup.loc[deceased_week_agegroup.week != '']
#deceased_week_agegroup['week'] = deceased_week_agegroup['week']

# adding y,week53 to y+1,week00
deceased_week_agegroup.loc[deceased_week_agegroup.loc[:,'week'] == 53, 'year'] += 1
deceased_week_agegroup.loc[deceased_week_agegroup.loc[:,'week'] == 53, 'week'] = 0

# set Index
deceased_week_agegroup.index = pd.MultiIndex.from_frame(deceased_week_agegroup[['year','week']])
deceased_week_agegroup.drop(columns=['year','week', 'Perioden'], inplace=True)
deceased_week_agegroup = deceased_week_agegroup.rename(columns={'Overledenen_1': 'Deceased'})

print('dataframe from cbs created, per week per agegroup: deceased_week_agegroup')
deceased_week_agegroup

dataframe from cbs created, per week per agegroup: deceased_week_agegroup


LeeftijdOp31December  Deceased
year week                               
1995 0           0 tot 65 jaar      62.0
     1           0 tot 65 jaar     476.0
     2           0 tot 65 jaar     511.0
     3           0 tot 65 jaar     473.0
     4           0 tot 65 jaar     513.0
...                        ...       ...
2021 8        80 jaar of ouder    1816.0
     9        80 jaar of ouder    1715.0
     10       80 jaar of ouder    1837.0
     11       80 jaar of ouder    1632.0
     12       80 jaar of ouder    1682.0

[4176 rows x 2 columns]

In [6]:
deceased_per_week = deceased_week_agegroup.groupby(['year','week']).sum()

print('dataframe from cbs created, per week: deceased_per_week')
deceased_per_week

dataframe from cbs created, per week: deceased_per_week


Deceased
year week          
1995 0        394.0
     1       2719.0
     2       2823.0
     3       2609.0
     4       2664.0
...             ...
2021 8       3198.0
     9       3086.0
     10      3228.0
     11      3001.0
     12      3086.0

[1387 rows x 1 columns]

In [7]:
# years only
deceased_year_agegroup = deceased[deceased['Perioden'].str.contains('week') == False]
deceased_year_agegroup = deceased_year_agegroup.rename(columns={'Perioden': 'year'})
deceased_year_agegroup.set_index('year', inplace=True)
deceased_year_agegroup.index = pd.PeriodIndex(deceased_year_agegroup.index, freq='Y')

print('dataframe from cbs created, per year per agegroup: deceased_year_agegroup')
deceased_year_agegroup

dataframe from cbs created, per year per agegroup: deceased_year_agegroup


,LeeftijdOp31December,Deceased
year,,
1995,0 tot 65 jaar,26183.0
1996,0 tot 65 jaar,26388.0
1997,0 tot 65 jaar,25715.0
1998,0 tot 65 jaar,26031.0
1999,0 tot 65 jaar,26557.0
...,...,...
2016,80 jaar of ouder,82349.0
2017,80 jaar of ouder,84095.0
2018,80 jaar of ouder,85820.0


In [8]:
deceased_per_year = deceased_year_agegroup.groupby('year').sum()

print('dataframe from cbs created, per year per agegroup: deceased_per_year')
deceased_per_year

dataframe from cbs created, per year per agegroup: deceased_per_year


,Deceased
year,
1995,135675.0
1996,137561.0
1997,135783.0
1998,137482.0
1999,140487.0
2000,140527.0
2001,140377.0
2002,142355.0
2003,141936.0


### Population & Deceased

In [9]:
# dropping unnececary columns
population.drop(columns=['ID','BevolkingAanHetBeginVanDePeriode_1','TotaleVestiging_4',
       'VestigingVanuitEenAndereGemeente_5', 'Immigratie_6',
       'TotaalVertrekInclAdmCorrecties_7', 'VertrekNaarAndereGemeente_8',
       'EmigratieInclusiefAdmCorrecties_9', 'OverigeCorrecties_10',
       'Bevolkingsgroei_11', 'BevolkingsgroeiRelatief_12',
       'BevolkingsgroeiSinds1Januari_13',
       'BevolkingsgroeiSinds1JanuariRela_14'], inplace=True)
population.rename(columns={"LevendGeborenKinderen_2": "Born",
                                        "Overledenen_3": "Deceased",
                                        "BevolkingAanHetEindeVanDePeriode_15": "Population"}, inplace=True)

In [10]:
# RegioS only, no nationals
population = population[population['RegioS'] != 'Nederland']
population = population[population['RegioS'].str.contains('LD') == False]
population = population[population['RegioS'].str.contains('PV') == False]
population = population[population['RegioS'].str.contains('CR') == False]

# Preparing periodes
population['year'] = population.loc[:,'Perioden'].str.slice(stop=4)
population['month'] = population.loc[:,'Perioden'].str.slice(start=5)
population = population.loc[population['month'] != '',:]
population.loc[:,'month'].replace({
    'januari': '01',
    'februari': '02',
    'maart': '03',
    'april': '04',
    'mei': '05',
    'juni': '06',
    'juli': '07',
    'augustus': '08',
    'september': '09',
    'oktober': '10',
    'november': '11',
    'december': '12'}, inplace=True)
population['Y-M'] = population['year'] + '-' + population['month']

population_month_municipality = population.groupby(['Y-M','RegioS']).sum()
population_month_municipality.reset_index(level='RegioS', inplace=True)
population_month_municipality.index = pd.PeriodIndex(population_month_municipality.index, freq='M')

print('dataframe from cbs created, per month per municipality: population_month_municipality')
population_month_municipality

dataframe from cbs created, per month per municipality: population_month_municipality


,RegioS,Born,Deceased,Population
Y-M,,,,
2002-01,'s-Gravendeel,9.0,7.0,8792.0
2002-01,'s-Gravenhage (gemeente),556.0,469.0,458474.0
2002-01,'s-Gravenzande,28.0,16.0,19426.0
2002-01,'s-Hertogenbosch,140.0,98.0,131731.0
2002-01,Aa en Hunze,25.0,21.0,25578.0
...,...,...,...,...
2021-02,Zutphen,27.0,43.0,48228.0
2021-02,Zwartewaterland,19.0,11.0,22836.0
2021-02,Zwijndrecht,44.0,35.0,44763.0


In [11]:
population_year_municipality = population.groupby(['year','RegioS']).sum()
population_year_municipality.reset_index(level='RegioS', inplace=True)
population_year_municipality.index = pd.PeriodIndex(population_year_municipality.index, freq='Y')

print('dataframe from cbs created, per year per municipality: population_year_municipality')
population_year_municipality

dataframe from cbs created, per year per municipality: population_year_municipality


,RegioS,Born,Deceased,Population
year,,,,
2002,'s-Gravendeel,102.0,66.0,106209.0
2002,'s-Gravenhage (gemeente),6328.0,4993.0,5534760.0
2002,'s-Gravenzande,253.0,136.0,233555.0
2002,'s-Hertogenbosch,1728.0,1119.0,1584919.0
2002,Aa en Hunze,289.0,251.0,306187.0
...,...,...,...,...
2021,Zutphen,55.0,78.0,96428.0
2021,Zwartewaterland,40.0,27.0,45666.0
2021,Zwijndrecht,71.0,88.0,89498.0


In [12]:
population_per_month = population.groupby('Y-M').sum()
population_per_month.index = pd.PeriodIndex(population_per_month.index, freq='M')

print('dataframe from cbs created, per month: population_per_month')
population_per_month

dataframe from cbs created, per month: population_per_month


,Born,Deceased,Population
Y-M,,,
2002-01,17019.0,13469.0,16112900.0
2002-02,15448.0,11735.0,16118953.0
2002-03,16792.0,13281.0,16124125.0
2002-04,15995.0,11968.0,16130246.0
2002-05,16800.0,11623.0,16136807.0
...,...,...,...
2020-10,14450.0,14603.0,17469635.0
2020-11,13512.0,14914.0,17474677.0
2020-12,13227.0,16801.0,17474693.0


In [13]:
population_per_year = population.groupby('year').sum()
deceased_per_year = deceased_year_agegroup.groupby('year').sum()
population_per_year = pd.concat([deceased_per_year,population_per_year])
population_per_year.index = pd.PeriodIndex(population_per_year.index, freq='Y')

print('dataframe from cbs created, per month: population_per_year')
population_per_year

dataframe from cbs created, per month: population_per_year


,Deceased,Born,Population
year,,,
1995,135675.0,NaN,NaN
1996,137561.0,NaN,NaN
1997,135783.0,NaN,NaN
1998,137482.0,NaN,NaN
1999,140487.0,NaN,NaN
2000,140527.0,NaN,NaN
2001,140377.0,NaN,NaN
2002,142355.0,NaN,NaN
2003,141936.0,NaN,NaN


### RIVM
#### RIVM per dag

In [14]:
prevalence = cleanup(prevalence)
prevalence = prevalence.groupby('Date').sum()
prevalence.index = pd.PeriodIndex(prevalence.index, freq='D')
rivm_per_day = prevalence

counts_per_municipality.rename(columns={'Date_of_publication':'Date',
                                        'Total_reported': 'Positive_tests',
                                        'Deceased': 'Deceased_covid'
                                       }, inplace=True)
counts_per_municipality = counts_per_municipality.groupby('Date').sum()
counts_per_municipality = counts_per_municipality[['Deceased_covid', 'Positive_tests', 'Hospital_admission']]
counts_per_municipality.index = pd.PeriodIndex(counts_per_municipality.index, freq='D')
rivm_per_day = rivm_per_day.merge(counts_per_municipality, on='Date', how='outer')

hospital_admission.rename(columns={'Date_of_statistics': 'Date',
                                   'Hospital_admission_notification': 'Hospital_admission_NICE_notification',
                                   'Hospital_admission': 'Hospital_admission_NICE'
                                  }, inplace=True)
hospital_admission = cleanup(hospital_admission)
hospital_admission = hospital_admission.groupby('Date').sum()
hospital_admission = hospital_admission[['Hospital_admission_NICE','Hospital_admission_NICE_notification']]
hospital_admission.index = pd.PeriodIndex(hospital_admission.index, freq='D')
rivm_per_day = rivm_per_day.merge(hospital_admission, on='Date', how='outer')

ic_admission.rename(columns={'Date_of_statistics': 'Date'}, inplace=True)
ic_admission = cleanup(ic_admission)
ic_admission = ic_admission.groupby('Date').sum()
ic_admission = ic_admission[['IC_admission','IC_admission_notification']]
ic_admission.index = pd.PeriodIndex(ic_admission.index, freq='D')
rivm_per_day = rivm_per_day.merge(ic_admission, on='Date', how='outer')

hospital_occupation.rename(columns={'Datum': 'Date'}, inplace=True)
hospital_occupation['Date'] = pd.to_datetime(hospital_occupation['Date'], format='%d-%m-%Y')
hospital_occupation['IC_Bedden_TOTAAL'] = hospital_occupation['IC_Bedden_COVID']+hospital_occupation['IC_Bedden_Non_COVID']
hospital_occupation = hospital_occupation.groupby('Date').sum()
hospital_occupation.index = pd.PeriodIndex(hospital_occupation.index, freq='D')
rivm_per_day = rivm_per_day.merge(hospital_occupation, on='Date', how='outer')

r = cleanup(r)
#r.rename(columns={'population': 'source_for_r'}, inplace=True)
r = r.groupby('Date').sum()
#r = r.groupby(['Date','source_for_r']).sum()
#r = r.reset_index(level='source_for_r')
r.index = pd.PeriodIndex(r.index, freq='D')
r = r.loc[r.index <= pd.Period(date.today()-timedelta(days=14), freq='D')]
rivm_per_day = rivm_per_day.merge(r, on='Date', how='outer')

homes_for_elderly.rename(columns={
    'Date_of_statistic_reported': 'Date',
    'Total_cases_reported': 'Positive_tests_elderly_homes',
    'Total_deceased_reported': 'Deceased_elderly_homes',
    'Total_new_infected_locations_reported': 'new_infected_elderly_homes',
    'Total_infected_locations_reported': 'balance_infected_elderly_homes'
    }, inplace=True)
homes_for_elderly = homes_for_elderly.groupby('Date').sum()
homes_for_elderly.index = pd.PeriodIndex(homes_for_elderly.index, freq='D')
homes_for_elderly = homes_for_elderly.loc['2020':]
rivm_per_day = rivm_per_day.merge(homes_for_elderly, on='Date', how='outer')


## Not yet implemented
# homes_for_disabled
# seventy_plus_living_at_home
# sewage
# support_and_behavior
# general_practioner

## STEP FOR ADDING
# rename columns if nececary
# cleanup if nececary
# groupby('Date').sum()
# re-order columns in nececeray
# apply periodIndex(df, freq='D')

#rivm_per_day = rivm_per_day.astype(int)

print('dataframe for RIVM data created, per day: rivm_per_day')
rivm_per_day

dataframe for RIVM data created, per day: rivm_per_day


,prev_low,prev_avg,prev_up,Deceased_covid,Positive_tests,Hospital_admission,Hospital_admission_NICE,Hospital_admission_NICE_notification,IC_admission,IC_admission_notification,...,IC_Nieuwe_Opnames_COVID,Kliniek_Nieuwe_Opnames_COVID,IC_Bedden_TOTAAL,Rt_low,Rt_avg,Rt_up,Positive_tests_elderly_homes,Deceased_elderly_homes,new_infected_elderly_homes,balance_infected_elderly_homes
Date,,,,,,,,,,,,,,,,,,,,,
2020-02-17,772.0,1189.0,1630.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.50,1.93,4.00,NaN,NaN,NaN,NaN
2020-02-18,899.0,1361.0,1801.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.00,1.74,4.00,NaN,NaN,NaN,NaN
2020-02-19,1076.0,1614.0,2151.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.50,1.59,2.83,NaN,NaN,NaN,NaN
2020-02-20,1301.0,1888.0,2480.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.83,1.74,2.83,NaN,NaN,NaN,NaN
2020-02-21,1595.0,2273.0,2962.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.67,2.03,3.67,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-02,107689.0,0.0,196276.0,16.0,7243.0,91.0,201.0,253.0,70.0,78.0,...,48.0,276.0,1124.0,NaN,NaN,NaN,21.0,2.0,3.0,194.0
2021-04-03,104152.0,0.0,198761.0,31.0,7635.0,67.0,155.0,157.0,45.0,43.0,...,54.0,243.0,1141.0,NaN,NaN,NaN,15.0,0.0,4.0,192.0
2021-04-04,100015.0,0.0,202692.0,16.0,6894.0,60.0,166.0,174.0,53.0,57.0,...,53.0,177.0,1097.0,NaN,NaN,NaN,18.0,1.0,2.0,188.0


#### RIVM per week

In [24]:
rivm_per_day['year'] = rivm_per_day.index.year
rivm_per_day['week'] = rivm_per_day.index.strftime('%W').astype(int)
rivm_per_week = rivm_per_day.groupby(by=['year', 'week']).sum()

print('dataframe for RIVM data created: rivm_per_week')
rivm_per_week

dataframe for RIVM data created: rivm_per_week


prev_low   prev_avg    prev_up  Deceased_covid  Positive_tests  \
year week                                                                   
2020 7       9937.0    14208.0    18502.0             0.0             0.0   
     8      43560.0    54395.0    64944.0             0.0             5.0   
     9     169542.0   203887.0   234483.0             2.0           158.0   
     10    386957.0   483165.0   568280.0            15.0           946.0   
     11    725369.0   903393.0  1066635.0           161.0          3076.0   
...             ...        ...        ...             ...             ...   
2021 10    572596.0   770714.0   967559.0           237.0         37077.0   
     11    662905.0   886755.0  1106112.0           207.0         44451.0   
     12    765053.0  1027353.0  1281629.0           189.0         51034.0   
     13    760115.0   153621.0  1378763.0           160.0         49892.0   
     14         0.0        0.0        0.0            28.0         10944.0   

           Hospital_admission  Hospital_admission_NICE  \
year week                                                
2020 7                    0.0                      0.0   
     8                    2.0                     17.0   
     9                   28.0                     95.0   
     10                 128.0                    434.0   
     11                 811.0                   1691.0   
...                       ...                      ...   
2021 10                 493.0                   1350.0   
     11                 458.0                   1518.0   
     12                 512.0                   1633.0   
     13                 503.0                   1588.0   
     14                 161.0                     64.0   

           Hospital_admission_NICE_notification  IC_admission  \
year week                                                       
2020 7                                      0.0           0.0   
     8                                      0.0           2.0   
     9                                      0.0          18.0   
     10                                     0.0         111.0   
     11                                     0.0         426.0   
...                                         ...           ...   
2021 10                                  1340.0         276.0   
     11                                  1477.0         318.0   
     12                                  1649.0         317.0   
     13                                  1787.0         376.0   
     14                                   212.0          33.0   

           IC_admission_notification  ...  IC_Nieuwe_Opnames_COVID  \
year week                             ...                            
2020 7                           0.0  ...                      0.0   
     8                           0.0  ...                      0.0   
     9                           0.0  ...                      0.0   
     10                          0.0  ...                      0.0   
     11                          0.0  ...                      0.0   
...                              ...  ...                      ...   
2021 10                        275.0  ...                    266.0   
     11                        315.0  ...                    280.0   
     12                        320.0  ...                    289.0   
     13                        401.0  ...                    344.0   
     14                         53.0  ...                    102.0   

           Kliniek_Nieuwe_Opnames_COVID  IC_Bedden_TOTAAL  Rt_low  Rt_avg  \
year week                                                                   
2020 7                              0.0               0.0    3.67   13.38   
     8                              0.0               0.0    9.68   14.39   
     9                              0.0               0.0   10.79   12.90   
     10                             0.0               0.0    8.52    9.43   
     11                            

In [25]:
#print('dataframe for RIVM data created, per week per municipality: ririvm_week_municipalityvm_per_week')
#rivm_week_municipality

#### RIVM per month

In [26]:
rivm_per_day['Y-M'] = rivm_per_day.index.strftime('%Y') + '-' + rivm_per_day.index.strftime('%m')
rivm_per_month = rivm_per_day.groupby('Y-M').sum()
rivm_per_day.drop(columns='Y-M', inplace=True)
rivm_per_month.index = pd.PeriodIndex(rivm_per_month.index, freq='M')
rivm_per_month = rivm_per_month[rivm_per_month.index < population_per_month.index.max()]

print('dataframe for RIVM data created, per month: rivm_per_month')
rivm_per_month

dataframe for RIVM data created, per month: rivm_per_month


,prev_low,prev_avg,prev_up,Deceased_covid,Positive_tests,Hospital_admission,Hospital_admission_NICE,Hospital_admission_NICE_notification,IC_admission,IC_admission_notification,...,IC_Bedden_TOTAAL,Rt_low,Rt_avg,Rt_up,Positive_tests_elderly_homes,Deceased_elderly_homes,new_infected_elderly_homes,balance_infected_elderly_homes,year,week
Y-M,,,,,,,,,,,,,,,,,,,,,
2020-02,42578.0,55366.0,68009.0,0.0,2.0,2.0,9.0,0.0,1.0,0.0,...,0.0,11.91,25.94,43.07,0.0,0.0,0.0,0.0,26260,97
2020-03,2327792.0,3032090.0,3695875.0,1004.0,12450.0,4578.0,6373.0,4125.0,1546.0,1404.0,...,0.0,31.80,36.69,41.76,1794.0,397.0,582.0,4620.0,62620,328
2020-04,1332819.0,1967587.0,2638830.0,3751.0,26613.0,6111.0,4846.0,6013.0,1112.0,1193.0,...,17913.0,15.87,22.12,28.92,5254.0,1882.0,459.0,22987.0,60600,448
2020-05,349830.0,528156.0,714653.0,1168.0,7138.0,997.0,904.0,1404.0,149.0,162.0,...,26972.0,8.44,24.68,45.25,1279.0,545.0,107.0,13075.0,62620,597
2020-06,82695.0,137417.0,195426.0,163.0,3853.0,154.0,205.0,233.0,47.0,50.0,...,18677.0,12.45,28.11,51.71,169.0,68.0,36.0,2745.0,60600,710
2020-07,113663.0,171298.0,230216.0,59.0,4184.0,84.0,166.0,520.0,31.0,40.0,...,18141.0,34.94,40.56,46.39,58.0,24.0,17.0,693.0,62620,868
2020-08,259211.0,374695.0,491638.0,75.0,16342.0,247.0,467.0,582.0,92.0,97.0,...,19087.0,31.09,33.78,36.48,226.0,51.0,52.0,1038.0,62620,1005
2020-09,1605283.0,2123109.0,2619147.0,185.0,50016.0,572.0,1692.0,1397.0,247.0,255.0,...,19892.0,36.15,37.64,39.05,1020.0,138.0,163.0,2670.0,60600,1104
2020-10,3361496.0,4572986.0,5777284.0,984.0,229608.0,2351.0,7043.0,6814.0,1166.0,1140.0,...,26366.0,30.82,31.50,32.22,5363.0,713.0,589.0,11515.0,62620,1275


In [27]:
#print('dataframe for RIVM data created, per month per municipality: ririvm_month_municipalityvm_per_month')
#rivm_month_municipality

#### RIVM per year

In [28]:
rivm_per_year = rivm_per_day.groupby(rivm_per_day.index.year).sum()
rivm_per_year.index.rename('year', inplace=True)
rivm_per_year.index = pd.PeriodIndex(rivm_per_year.index, freq='Y')
rivm_per_year = rivm_per_year[rivm_per_year.index < population_per_year.index.max()]

print('dataframe for RIVM data created, per month: rivm_per_month')
rivm_per_year

dataframe for RIVM data created, per month: rivm_per_month


,prev_low,prev_avg,prev_up,Deceased_covid,Positive_tests,Hospital_admission,Hospital_admission_NICE,Hospital_admission_NICE_notification,IC_admission,IC_admission_notification,...,IC_Bedden_TOTAAL,Rt_low,Rt_avg,Rt_up,Positive_tests_elderly_homes,Deceased_elderly_homes,new_infected_elderly_homes,balance_infected_elderly_homes,year,week
year,,,,,,,,,,,,,,,,,,,,,
2020,14994635.0,20522343.0,26031965.0,11401.0,795694.0,20035.0,35133.0,34610.0,6578.0,6541.0,...,210074.0,274.14,343.17,428.48,29278.0,6091.0,3068.0,98619.0,644380,9343


In [29]:
#print('dataframe for RIVM data created, per month per municipality: ririvm_month_municipalityvm_per_month')
#rivm_month_municipality

## Merging CBS and RIVM
### Per week deceased_per_week + rivm_per_week

In [34]:
per_week = pd.merge(deceased_per_week, rivm_per_week, left_index=True, right_index=True, how='outer', sort=True)

print('dataframe CBS and RIVM combined, per week: per_week')
per_week

dataframe CBS and RIVM combined, per week: per_week


Deceased  prev_low   prev_avg    prev_up  Deceased_covid  \
year week                                                             
1995 0        394.0       NaN        NaN        NaN             NaN   
     1       2719.0       NaN        NaN        NaN             NaN   
     2       2823.0       NaN        NaN        NaN             NaN   
     3       2609.0       NaN        NaN        NaN             NaN   
     4       2664.0       NaN        NaN        NaN             NaN   
...             ...       ...        ...        ...             ...   
2021 10      3228.0  572596.0   770714.0   967559.0           237.0   
     11      3001.0  662905.0   886755.0  1106112.0           207.0   
     12      3086.0  765053.0  1027353.0  1281629.0           189.0   
     13         NaN  760115.0   153621.0  1378763.0           160.0   
     14         NaN       0.0        0.0        0.0            28.0   

           Positive_tests  Hospital_admission  Hospital_admission_NICE  \
year week                                                                
1995 0                NaN                 NaN                      NaN   
     1                NaN                 NaN                      NaN   
     2                NaN                 NaN                      NaN   
     3                NaN                 NaN                      NaN   
     4                NaN                 NaN                      NaN   
...                   ...                 ...                      ...   
2021 10           37077.0               493.0                   1350.0   
     11           44451.0               458.0                   1518.0   
     12           51034.0               512.0                   1633.0   
     13           49892.0               503.0                   1588.0   
     14           10944.0               161.0                     64.0   

           Hospital_admission_NICE_notification  IC_admission  ...  \
year week                                                      ...   
1995 0                                      NaN           NaN  ...   
     1                                      NaN           NaN  ...   
     2                                      NaN           NaN  ...   
     3                                      NaN           NaN  ...   
     4                                      NaN           NaN  ...   
...                                         ...           ...  ...   
2021 10                                  1340.0         276.0  ...   
     11                                  1477.0         318.0  ...   
     12                                  1649.0         317.0  ...   
     13                                  1787.0         376.0  ...   
     14                                   212.0          33.0  ...   

           IC_Nieuwe_Opnames_COVID  Kliniek_Nieuwe_Opnames_COVID  \
year week                                                          
1995 0                         NaN                           NaN   
     1                         NaN                           NaN   
     2                         NaN                           NaN   
     3                         NaN                           NaN   
     4                         NaN                           NaN   
...                            ...                           ...   
2021 10                      266.0                        1403.0   
     11                      280.0                        1500.0   
     12                      289.0                        1635.0   
     13                      344.0                        1710.0   
     14                      102.0                         412.0   

           IC_Bedden_TOTAAL  Rt_low  Rt_avg  Rt_up  \
year week                                            
1995 0                  NaN     NaN     NaN    NaN   
     1                  NaN     NaN     NaN    NaN   
     2                  NaN     NaN     NaN    NaN   
     3                  NaN     NaN     NaN    NaN   
    

In [35]:
#print('dataframe CBS and RIVM combined, per week per municipality: week_municipality')
#week_municipality

### Per month population_per_month + rivm_per_month

In [36]:
per_month = pd.merge(population_per_month, rivm_per_month, on='Y-M', how='outer', sort=True)

print('dataframe CBS and RIVM combined, per month: per_month')
per_month

#print('dataframe CBS and RIVM combined, per month per municipality: month_municipality')
#month_municipality

dataframe CBS and RIVM combined, per month: per_month


,Born,Deceased,Population,prev_low,prev_avg,prev_up,Deceased_covid,Positive_tests,Hospital_admission,Hospital_admission_NICE,...,IC_Bedden_TOTAAL,Rt_low,Rt_avg,Rt_up,Positive_tests_elderly_homes,Deceased_elderly_homes,new_infected_elderly_homes,balance_infected_elderly_homes,year,week
Y-M,,,,,,,,,,,,,,,,,,,,,
2002-01,17019.0,13469.0,16112900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-02,15448.0,11735.0,16118953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-03,16792.0,13281.0,16124125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-04,15995.0,11968.0,16130246.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-05,16800.0,11623.0,16136807.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10,14450.0,14603.0,17469635.0,3361496.0,4572986.0,5777284.0,984.0,229608.0,2351.0,7043.0,...,26366.0,30.82,31.50,32.22,5363.0,713.0,589.0,11515.0,62620.0,1275.0
2020-11,13512.0,14914.0,17474677.0,2414778.0,3354978.0,4306965.0,1964.0,171998.0,2485.0,6036.0,...,31179.0,29.65,30.47,31.32,4933.0,973.0,393.0,19225.0,60600.0,1365.0
2020-12,13227.0,16801.0,17474693.0,3104490.0,4204661.0,5293922.0,2048.0,273490.0,2454.0,7392.0,...,31847.0,31.02,31.68,32.31,9182.0,1300.0,670.0,20051.0,62620.0,1546.0


### Per month population_per_year + rivm_per_year

In [37]:
per_year = pd.merge(population_per_year.drop_duplicates(subset=['Deceased'], keep='last'), rivm_per_year, left_index=True, right_index=True, how='outer', sort=True, copy=False, validate='m:1')
per_year = per_year[(per_year['Born'].isnull() * 1 + (1 - per_year['prev_low'].isnull() * 1)) != 2]

print('dataframe CBS and RIVM combined, per month: per_year')
per_year

#print('dataframe CBS and RIVM combined, per year per municipality: year_municipality')
#year_municipality

dataframe CBS and RIVM combined, per month: per_year


,Deceased,Born,Population,prev_low,prev_avg,prev_up,Deceased_covid,Positive_tests,Hospital_admission,Hospital_admission_NICE,...,IC_Bedden_TOTAAL,Rt_low,Rt_avg,Rt_up,Positive_tests_elderly_homes,Deceased_elderly_homes,new_infected_elderly_homes,balance_infected_elderly_homes,year,week
year,,,,,,,,,,,,,,,,,,,,,
1995,135675.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,137561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,135783.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,137482.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999,140487.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000,140527.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001,140377.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002,142355.0,202083.0,193809414.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003,141936.0,200297.0,194705905.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
